In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Writing setup.sh


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install seqeval
!sh setup.sh

!cp /content/drive/My\ Drive/fincausal/* ./
!git clone https://github.com/huggingface/transformers/

In [ ]:
!export BERT_MODEL=bert-large-cased &&\
export MAX_LENGTH=256 &&\
python3 ./transformers/examples/token-classification/scripts/\
preprocess.py bio_test_task2.csv $BERT_MODEL $MAX_LENGTH > test.txt

2020-06-24 17:20:59.288975: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
!cp bio_train_task2.csv train.txt
!cp bio_dev_task2.csv dev.txt
!cat train.txt | cut -d " " -f 2 | grep -v "^$"| sort | uniq > labels.txt
!cat labels.txt

0
B-C
B-E
I-C
I-E


In [ ]:
with open("test.txt") as f:
    predictions = f.readlines()
predictions = [p.strip().split() for p in predictions]

sents = []
sent = []
for p in predictions:
    if not p:
        sents.append(sent)
        sent = []
    else:
        sent.append(p)
print(len(sents))

638


In [ ]:
# --do_train \
# --do_eval \
!export MAX_LENGTH=256 &&\
export BERT_MODEL=bert-large-cased &&\
export OUTPUT_DIR=fincausal2 &&\
export BATCH_SIZE=16 &&\
export NUM_EPOCHS=3 &&\
export SAVE_STEPS=5000 &&\
export SEED=1 &&\
python3 ./transformers/examples/token-classification/run_ner.py --data_dir ./ \
--labels ./labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--num_train_epochs $NUM_EPOCHS \
--per_device_train_batch_size $BATCH_SIZE \
--save_steps $SAVE_STEPS \
--seed $SEED \
--do_predict \
--overwrite_output_dir \
--fp16

2020-06-24 17:22:36.176840: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
06/24/2020 17:22:37 - INFO - transformers.training_args -   PyTorch: setting up devices
06/24/2020 17:22:37 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
06/24/2020 17:22:37 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='fincausal2', overwrite_output_dir=True, do_train=False, do_eval=False, do_predict=True, evaluate_during_training=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=5000, save_total_limit=None, no_cuda=False, se